In [5]:
import pandas
import os

In [9]:
models = ["gpt-4o", "gpt-4o-mini", "llama3.1-8b-instruct-q4_0"]
setting = ["", "_no_name"]
result_path = "../results/llms"

In [12]:
# load results
results = {}
for model in models:
    for s in setting:
        result_file = os.path.join(result_path, f"{model}{s}_validation.csv")
        results[f"{model}{s}"] = pandas.read_csv(result_file)

In [13]:
accuracy = {}

# calculate accuracy
for model in models:
    for s in setting:
        result = results[f"{model}{s}"]
        accuracy[f"{model}{s}"] = result["valid?"].sum() / len(result)

In [15]:
accuracy

{'gpt-4o': 0.6363636363636364,
 'gpt-4o_no_name': 0.6363636363636364,
 'gpt-4o-mini': 0.6666666666666666,
 'gpt-4o-mini_no_name': 0.6363636363636364,
 'llama3.1-8b-instruct-q4_0': 0.45454545454545453,
 'llama3.1-8b-instruct-q4_0_no_name': 0.42424242424242425}